In [1]:
import scraper as s
import pandas as pd
import numpy as np
import datetime

In [2]:
op = s.getYahooOptions("AAPL", 1600387200)

In [3]:
op['strike-date'] = pd.to_datetime(op['strike-date'], unit='s')
op['strike'] = pd.to_numeric(op['strike'],errors='coerce')
op['last'] = pd.to_numeric(op['last'],errors='coerce')
op['bid'] = pd.to_numeric(op['bid'],errors='coerce')
op['ask'] = pd.to_numeric(op['ask'],errors='coerce')
op['volume'] = pd.to_numeric(op['volume'],errors='coerce')
op['date'] = pd.to_datetime(op['date'], "%Y-%m-%d %H:%M:%S")
op['strike-date'] = pd.to_datetime(op['strike-date'], "%Y-%m-%d %H:%M:%S")
op['last_trade'] = pd.to_datetime(op['last_trade'], "%Y-%m-%d")
op = op.set_index('date')

In [4]:
op.head()

,type,in-money,contract,strike-date,last_trade,strike,last,bid,ask,volume,iv
date,,,,,,,,,,,
2020-08-11 20:23:09.024993,Call,in-the-money,AAPL200918C00095000,2020-09-18,2020-08-10,95.0,345.46,349.90,351.85,1.0,250.95%
2020-08-11 20:23:09.024993,Call,in-the-money,AAPL200918C00100000,2020-09-18,2020-08-06,100.0,354.20,344.85,346.90,262.0,242.77%
2020-08-11 20:23:09.024993,Call,in-the-money,AAPL200918C00105000,2020-09-18,2020-08-10,105.0,338.14,339.95,341.80,1.0,235.01%
2020-08-11 20:23:09.024993,Call,in-the-money,AAPL200918C00110000,2020-09-18,2020-08-06,110.0,345.80,335.10,336.75,2.0,228.76%
2020-08-11 20:23:09.024993,Call,in-the-money,AAPL200918C00115000,2020-09-18,2020-06-23,115.0,255.30,254.10,258.90,2.0,0.00%


In [5]:
apple = op.drop(['volume', 'iv', 'in-money', 'bid', 'ask','last_trade', 'contract'], axis=1)
apple = apple.reset_index()
apple = apple.set_index(['strike-date', 'type'])
apple['date'] = pd.to_datetime(apple['date']).dt.strftime('%m-%d-%Y')
apple['date'] = pd.to_datetime(apple['date'])
strike_pivot = apple.pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
strike_cols = strike_pivot.columns

In [14]:
new_apple = []
apple2 = apple

apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type','strike-date'])
apple_pivot_call = apple2.loc[('Call', cols)]
strike_pivot = apple_pivot_call.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_cols = strike_pivot.columns
apple_pivot_put = apple2.loc[('Put', cols)]
strike_pivot = apple_pivot_put.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_puts = strike_pivot.columns
    
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type', 'strike','strike-date'])

i = 0
j = 0
aCLo = apple2.loc[('Call', strike_value_cols[i], cols)]
aCLo = aCLo.reset_index()
#aCLo = aCLo.drop(['type', 'strike-date'], axis=1)
aCLo.rename(columns = {'last':'priceCLo', 'strike': 'strikeCLo'}, inplace = True)
aCHi = apple2.loc[('Call', strike_value_cols[i+1], cols)]
aCHi = aCHi.reset_index()
#aCHi = aCHi.drop(['type', 'strike-date'], axis=1)
aCHi.rename(columns = {'last':'priceCHi', 'strike': 'strikeCHi'}, inplace = True)
aPLo = apple2.loc[('Put', strike_value_puts[j], cols)]
aPLo = aPLo.reset_index()
#aPLo = aPLo.drop(['type', 'strike-date'], axis=1)
aPLo.rename(columns = {'last':'pricePLo', 'strike': 'strikePLo'}, inplace = True)
aPHi = apple2.loc[('Put', strike_value_puts[j+1], cols)]
aPHi = aPHi.reset_index()
#aPHi = aPHi.drop(['type', 'strike-date'], axis=1)
aPHi.rename(columns = {'last':'pricePHi', 'strike': 'strikePHi'}, inplace = True)

a = pd.merge(aCLo, aCHi, on='date')
a = pd.merge(a, aPLo, on='date')
a = pd.merge(a, aPHi, on='date')

new_apple.append(a)

print(a)


KeyError: 'date'

In [25]:
aPLo.iloc[1,1]

0.01

In [ ]:
new_apple = []
apple2 = apple
#for cols in strike_cols:
cols = strike_cols[0]
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type','strike-date'])
apple_pivot_call = apple2.loc[('Call', cols)]
strike_pivot = apple_pivot_call.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_cols = strike_pivot.columns
apple_pivot_put = apple2.loc[('Put', cols)]
strike_pivot = apple_pivot_put.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_puts = strike_pivot.columns
    
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type', 'strike','strike-date'])

for i in range(len(strike_value_cols)-1):
    for j in range(len(strike_value_puts)-1):
        aCLo = apple2.loc[('Call', strike_value_cols[i], cols)]
        aCLo = aCLo.reset_index()
        #aCLo = aCLo.drop(['type', 'strike-date'], axis=1)
        aCLo.rename(columns = {'last':'priceCLo', 'strike': 'strikeCLo'}, inplace = True)
        aCHi = apple2.loc[('Call', strike_value_cols[i+1], cols)]
        aCHi = aCHi.reset_index()
        #aCHi = aCHi.drop(['type', 'strike-date'], axis=1)
        aCHi.rename(columns = {'last':'priceCHi', 'strike': 'strikeCHi'}, inplace = True)
        aPLo = apple2.loc[('Put', strike_value_puts[j], cols)]
        aPLo = aPLo.reset_index()
        #aPLo = aPLo.drop(['type', 'strike-date'], axis=1)
        aPLo.rename(columns = {'last':'pricePLo', 'strike': 'strikePLo'}, inplace = True)
        aPHi = apple2.loc[('Put', strike_value_puts[j+1], cols)]
        aPHi = aPHi.reset_index()
        #aPHi = aPHi.drop(['type', 'strike-date'], axis=1)
        aPHi.rename(columns = {'last':'pricePHi', 'strike': 'strikePHi'}, inplace = True)

        a = pd.merge(aCLo, aCHi, on='date')
        a = pd.merge(a, aPLo, on='date')
        a = pd.merge(a, aPHi, on='date')

        new_apple.append(a)